In [3]:
import random
import time
from IPython.display import clear_output
from itertools import cycle
import copy

In [4]:
def combinar_dados_em_opcoes(lista):
    combinacoes = set()

    combinacoes.add((lista[0] + lista[1],lista[2] + lista[3]))
    combinacoes.add((lista[0] + lista[2],lista[1] + lista[3]))
    combinacoes.add((lista[0] + lista[3],lista[1] + lista[2]))

    return list(combinacoes)


In [33]:
class Jogador():
    def __init__(self,nome):
        self.nome=nome
    
    def decidir(self, valores_possiveis, tabuleiro):
        if (len(tabuleiro.lista_colunas_com_marcadores()) == 3):
            deseja_continuar = False
        else:
            deseja_continuar = True
        for opcao in valores_possiveis:
            if tabuleiro.dados_podem_avancar(opcao):
                return Decisao(deseja_continuar, opcao)


In [6]:
class Decisao():
    def __init__(self,deseja_continuar, valores_escolhidos):
        self.deseja_continuar = deseja_continuar
        self.valores_escolhidos = valores_escolhidos

In [7]:
class Coluna():
    def __init__(self,comprimento, lista_jogadores):
        self.comprimento=comprimento
        self.marcador=0
        self.bloqueado=False

        self.posicao=dict()
        self.posicao[lista_jogadores[0]] = 0
        self.posicao[lista_jogadores[1]] = 0

    def avancar_marcador(self):
        self.marcador+=1

    def gravar_avanco(self,jogador_ativo):
        self.posicao[jogador_ativo] = self.marcador
        self.marcador_reset()
        if self.posicao[jogador_ativo] > self.comprimento:
            self.bloquear_coluna()

    def marcador_reset(self):
        self.marcador = 0

    def marcador_start(self,jogador_ativo):
        self.marcador = self.posicao[jogador_ativo] + 1

    def possui_marcador(self):
        return self.marcador > 0
    
    def bloquear_coluna(self):
        self.bloqueado=True
    
    def esta_bloqueado(self):
        return self.bloqueado

In [45]:

class Tabuleiro():
    def __init__(self, configuracao_jogo):
        self.configuracao_jogo = configuracao_jogo
        self.colunas={}
        for i in range(11):
            self.colunas[i+2] = Coluna(configuracao_jogo.comprimento_colunas[i], configuracao_jogo.lista_jogadores)
    def __str__(self):
        string_retorno = 'Status tabuleiro\n'
        for coluna in self.colunas:
            string_retorno += '[' + str(coluna) + ']' + 'marcador: ' + str(self.colunas[coluna].marcador)
            string_retorno += ' comprimento: ' + str(self.colunas[coluna].comprimento)
            for jogador in self.configuracao_jogo.lista_jogadores:
                string_retorno += ' - ' + str(jogador.nome) + ' ' + str(self.colunas[coluna].posicao[jogador])
            string_retorno += '\n'
        return string_retorno
    
    def rolar_dados(self):
        return combinar_dados_em_opcoes([random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7),random.randrange(1, 7)])

    def lista_colunas_com_marcadores(self):
        return list(filter( lambda coluna : self.colunas[coluna].possui_marcador(), self.colunas))
    
    def lista_colunas_bloqueadas(self):
        return list(filter( lambda coluna : self.colunas[coluna].esta_bloqueado(), self.colunas))

    def limpar_tabuleiro(self):
        for coluna in self.colunas:
            self.colunas[coluna].marcador_reset()

    def gravar_avancos(self , jogador_ativo):
        for coluna in self.lista_colunas_com_marcadores():
            self.colunas[coluna].gravar_avanco(jogador_ativo)

    def pode_adicionar_marcador_na_coluna(self, valor):
        return (valor not in self.lista_colunas_com_marcadores() 
                and valor not in self.lista_colunas_bloqueadas()
                and len(self.lista_colunas_com_marcadores()) < self.configuracao_jogo.quantidade_marcadores)

    def avancar_marcador_coluna(self, valor):
        self.colunas[valor].avancar_marcador()
       
    def start_marcador_coluna(self, valor , jogador_ativo):
        self.colunas[valor].marcador_start(jogador_ativo)

    def valores_podem_avancar(self, valores_possiveis):
        for valores in valores_possiveis:
            if self.dados_podem_avancar(valores):
                return True
        return False

    def dados_podem_avancar(self, dados):
        for valor in dados:
            if (self.pode_adicionar_marcador_na_coluna(valor)
                or self.colunas[valor].possui_marcador()):
                return True
        return False

    def valida_decisao(self, decisao, valores_possiveis):
        if decisao.valores_escolhidos not in valores_possiveis:
            return False
        
        return self.dados_podem_avancar(decisao.valores_escolhidos)

    def executa_decisao(self, decisao, jogador_ativo):
        for valor in decisao.valores_escolhidos:
            if self.pode_adicionar_marcador_na_coluna(valor):
                self.start_marcador_coluna(valor , jogador_ativo)
            elif self.colunas[valor] in self.lista_colunas_com_marcadores():
                self.avancar_marcador_coluna(valor)

        if not decisao.deseja_continuar:
            self.gravar_avancos(jogador_ativo)

    def pode_encerrar(self):
        return len(self.lista_colunas_com_marcadores) < self.configuracao_jogo.quantidade_marcadores



In [9]:
class Configuracao_jogo():
    def __init__(self, quantidade_marcadores , lista_jogadores , comprimento_colunas):
        self.quantidade_marcadores = quantidade_marcadores
        self.lista_jogadores = lista_jogadores
        self.comprimento_colunas = comprimento_colunas

In [43]:
class Gerenciador_partida():
    def __init__(self, configuracao_jogo):
        self.configuracao_jogo = configuracao_jogo

        self.tabuleiro = Tabuleiro(configuracao_jogo)

    def iniciar_jogo(self):
        lista_circular_jogadores = cycle(self.configuracao_jogo.lista_jogadores)
        jogador_ativo = next(lista_circular_jogadores)

        while (len(self.tabuleiro.lista_colunas_bloqueadas()) < 5):
            valores_possiveis = self.tabuleiro.rolar_dados()
            print('---- Inicio da jogada ----')
            print('jogador_ativo')
            print(jogador_ativo.nome)
            print(self.tabuleiro)
            
            if self.tabuleiro.valores_podem_avancar(valores_possiveis):
                decisao = jogador_ativo.decidir(valores_possiveis, copy.deepcopy(self.tabuleiro)) ##TODO
                print('decisao.valores_escolhidos')
                print(decisao.valores_escolhidos)

                if (self.tabuleiro.valida_decisao(decisao, valores_possiveis)):
                    self.tabuleiro.executa_decisao(decisao , jogador_ativo)
                    if not decisao.deseja_continuar:
                        print('jogador escolheu encerrar')
                    if self.tabuleiro.pode_encerrar and not decisao.deseja_continuar :
                        print('jogador encerrou')
                        jogador_ativo = next(lista_circular_jogadores)
                else:
                    raise Exception("Valor escolhido não era uma opção")
            else:
                print('jogador perdeu a vez')
                self.tabuleiro.limpar_tabuleiro()
                jogador_ativo = next(lista_circular_jogadores)
        



In [46]:
comprimento_colunas = [2,4,6,8,10,12,10,8,6,4,2]
lista_jogadores = [Jogador('Gustavo'), Jogador('Pla')]

configuracao_jogo = Configuracao_jogo( 3, lista_jogadores , comprimento_colunas)

gerenciador = Gerenciador_partida(configuracao_jogo) 

gerenciador.iniciar_jogo()


---- Inicio da jogada ----
jogador_ativo
Gustavo
Status tabuleiro
[2]marcador: 0comprimento: 2 - Gustavo 0 - Pla 0
[3]marcador: 0comprimento: 4 - Gustavo 0 - Pla 0
[4]marcador: 0comprimento: 6 - Gustavo 0 - Pla 0
[5]marcador: 0comprimento: 8 - Gustavo 0 - Pla 0
[6]marcador: 0comprimento: 10 - Gustavo 0 - Pla 0
[7]marcador: 0comprimento: 12 - Gustavo 0 - Pla 0
[8]marcador: 0comprimento: 10 - Gustavo 0 - Pla 0
[9]marcador: 0comprimento: 8 - Gustavo 0 - Pla 0
[10]marcador: 0comprimento: 6 - Gustavo 0 - Pla 0
[11]marcador: 0comprimento: 4 - Gustavo 0 - Pla 0
[12]marcador: 0comprimento: 2 - Gustavo 0 - Pla 0

decisao.valores_escolhidos
(5, 9)
---- Inicio da jogada ----
jogador_ativo
Gustavo
Status tabuleiro
[2]marcador: 0comprimento: 2 - Gustavo 0 - Pla 0
[3]marcador: 0comprimento: 4 - Gustavo 0 - Pla 0
[4]marcador: 0comprimento: 6 - Gustavo 0 - Pla 0
[5]marcador: 1comprimento: 8 - Gustavo 0 - Pla 0
[6]marcador: 0comprimento: 10 - Gustavo 0 - Pla 0
[7]marcador: 0comprimento: 12 - Gustavo 0 

In [13]:
def rodar_jogo(Configuracao_jogo):
    #preparando as classes
    tabuleiro=Tabuleiro(Configuracao_jogo)
    pontos=[]
    lista_jogadores = Configuracao_jogo.lista_jogadores
    jogador_ativo = lista_jogadores[0]
    
    #choose your move
    def executar_decisao(decisao_do_jogador , jogador_ativo):
        '''execute the chosen move'''
        #make your move
        executou_movimento=False
        #### #for valor in decisao_do_jogador:
        while(decisao_do_jogador!=[]):
            for i in range (2,13): ###acho que podemos percorrer o decisao_do_jogador ao invés de range(2,13).
                if decisao_do_jogador==[]:
                    break
                elif tabuleiro.pode_adicionar_marcador_na_coluna(decisao_do_jogador[0]): #se tem marcardor disponível, bota marcador
                    print(f'Colocou marcador na coluna: {decisao_do_jogador[0]}')
                    print(decisao_do_jogador[0], tabuleiro.colunas[decisao_do_jogador[0]].marcador)

                    tabuleiro.start_marcador_coluna(decisao_do_jogador[0],jogador_ativo)
                    decisao_do_jogador.remove(decisao_do_jogador[0])
                    executou_movimento=True

                elif decisao_do_jogador[0] in tabuleiro.lista_colunas_com_marcadores(): #se tem marcador posicionado, avança marcador
                    print(f'Avançou marcador na coluna: {decisao_do_jogador[0]}')
                    print(decisao_do_jogador[0], tabuleiro.colunas[decisao_do_jogador[0]].marcador)

                    tabuleiro.avancar_marcador_coluna(decisao_do_jogador[0])
                    decisao_do_jogador.remove(decisao_do_jogador[0])
                    executou_movimento=True
                
                else:
                    print('')
                    print('')
                    decisao_do_jogador.remove(decisao_do_jogador[0])
                    
        if executou_movimento==False:
            tabuleiro.limpar_tabuleiro()
            return False
        return True
                    
    def passar_turno(jogador_ativo):
        '''voluntarily end your turn'''
        tabuleiro.gravar_avancos(jogador_ativo)
            
    for i in range(500):
        
        time.sleep(0.1)
        clear_output(wait=True)
        dados=tabuleiro.rolar_dados()
        decisao_do_jogador=list(dados[random.randrange(0,len(dados))]) ### Nessa versão estamos escolhendo uma opção ao acaso ###
        print(f'jogador {jogador_ativo.nome}')
        print(f'jogador {jogador_ativo.nome} escolheu: {decisao_do_jogador}')
        if not executar_decisao(decisao_do_jogador , jogador_ativo): ### a jogada deu ruim, o jogador ativo encerra o turno.
            print(f'jogador {jogador_ativo.nome} Explodiu')
            if jogador_ativo==lista_jogadores[0]: 
                jogador_ativo=lista_jogadores[1]
            else:
                jogador_ativo=lista_jogadores[0]            
        elif len(tabuleiro.lista_colunas_com_marcadores())>=3:
            passar_turno(jogador_ativo)
            print(f'jogador {jogador_ativo.nome} Passou o turno')
            if jogador_ativo==lista_jogadores[0]:
                jogador_ativo=lista_jogadores[1]
            else:
                jogador_ativo=lista_jogadores[0]
        print(f'Tabuleiro: \ncolunas com marcadores: {tabuleiro.lista_colunas_com_marcadores()}')
        print(f'Colunas Bloqueadas: {tabuleiro.lista_colunas_bloqueadas()}')
        print(f'pontos:: {pontos}')
        for i in range (2,13):
            print(f'Coluna {i}: {list(map(lambda jogador: tabuleiro.colunas[i].posicao[jogador], tabuleiro.colunas[i].posicao))}')
        if len(tabuleiro.lista_colunas_bloqueadas())>=5:
            print(tabuleiro.lista_colunas_bloqueadas())
            break

In [14]:
n_colunas = [2,4,6,8,10,12,10,8,6,4,2]
lista_jogadores = [Jogador('Gustavo') , Jogador('Plá')]
configuracoes = Configuracao_jogo(3 , lista_jogadores , n_colunas)

rodar_jogo(configuracoes)

jogador Plá
jogador Plá escolheu: [4, 6]
Colocou marcador na coluna: 4
4 0


jogador Plá Passou o turno
Tabuleiro: 
colunas com marcadores: []
Colunas Bloqueadas: [2, 5, 6, 8, 10]
pontos:: []
Coluna 2: [1, 3]
Coluna 3: [2, 0]
Coluna 4: [5, 3]
Coluna 5: [11, 6]
Coluna 6: [10, 11]
Coluna 7: [7, 11]
Coluna 8: [12, 3]
Coluna 9: [8, 5]
Coluna 10: [6, 7]
Coluna 11: [4, 4]
Coluna 12: [1, 2]
[2, 5, 6, 8, 10]
